# Animating the Minimization of Finite State Machines

This notebook generates and displays the animation of the minimization of an FSM. After executing each cell, you will be able to view the animation at the end of the notebook.

Import `gvanim`, which was installed using `pip install GraphvizAnim`. This module is used to generate the interactive animations. 

In [1]:
from gvanim import Animation
from gvanim.jupyter import interactive
ga_det = Animation() # variable used to represent the animation 

In the section below, we introduce all the classes and methods needed to produce a Deterministic Finite State Machine. These methods were referenced from the lecture notes (02 Regular Languages).

In [2]:
class FiniteStateMachine:
    def __init__(self, T, Q, R, q0, F):
        self.T, self.Q, self.R, self.q0, self.F = T, Q, R, q0, F
    def __repr__(self):
        return str(self.q0) + '\n' + ' '.join(self.F) + '\n' + \
               '\n'.join(r[0] + ' ' + r[1] + ' → ' + r[2] for r in self.R)

def parseFSM(fsm: str) -> FiniteStateMachine:
    fsm = [line for line in fsm.split('\n') if line.strip() != '']
    q0 = fsm[0].split()[0] # first line: initialstate
    F = set(fsm[1].split()) # second line: finalstate, finalstate, ...
    R = set()
    for line in fsm[2:]: # all subsequent lines: "source symbol → target"
        l, r = line.split('→')
        R |= {(l.split()[0], l.split()[1], r.split()[0])}
    T = {r[1] for r in R}
    Q = {q0} | F | {r[0] for r in R} | {r[2] for r in R}
    return FiniteStateMachine(T, Q, R, q0, F)

class Choice:
    def __init__(self, e1, e2): self.e1, self.e2 = e1, e2
class Conc:
    def __init__(self, e1, e2): self.e1, self.e2 = e1, e2
class Star:
    def __init__(self, e): self.e = e
        
def syntaxgraph(re):
    global node, T
    if re == '': return {(None, None)}
    elif type(re) == str:
        node += 1; T.add(re); return {(None, (re, str(node))), ((re, str(node)), None)}
    elif type(re) == Choice:
        return syntaxgraph(re.e1) | syntaxgraph(re.e2)
    elif type(re) == Conc:
        g1, g2 = syntaxgraph(re.e1), syntaxgraph(re.e2)
        return {(a, b) for (a, b) in g1 if b} | \
               {(a, b) for (a, b) in g2 if a} | \
               {(a, b) for (a, c) in g1 for (d, b) in g2 if not c and not d}
    elif type(re) == Star:
        g = syntaxgraph(re.e)
        return {(None, None)} | g | \
               {(a, b) for (a, c) in g for (d, b) in g if not c and not d}
    else: raise Exception('not a regular expression')
        
def convertRegExToFSM(re):
    global node, T; node, T = 0, set()
    g = syntaxgraph(re)
    Q = {str(n) for n in range(node + 1)}
    R = {('0', b[0], b[1]) for (a, b) in g if not a and b} | \
        {(a[1], b[0], b[1]) for (a, b) in g if a and b}
    F = {a[1] for (a, b) in g if a and not b} | ({'0'} if (None, None) in g else set())
    return FiniteStateMachine(T, Q, R, '0', F)

def string(s: set) -> str:
    return '{' + ', '.join(e for e in s) + '}'

def deterministicFSM(fsm: FiniteStateMachine) -> FiniteStateMachine:
    qq0 = string({fsm.q0})
    QQ, RR, visited = {qq0}, set(), set()
    #print(QQ, RR, visited)
    while visited != QQ:
        qq = (QQ - visited).pop(); visited |= {qq}
        for t in fsm.T:
            rr = {r for (q, u, r) in fsm.R if u == t and q in qq}
            if rr != set(): QQ |= {string(rr)}; RR |= {(qq, t, string(rr))}
        #print(QQ, RR, visited)
    FF = {qq for qq in QQ for f in fsm.F if f in qq}
    output = FiniteStateMachine(fsm.T, QQ, RR, qq0, FF)
    output = str(output)
    return output

###### Defining the Nondeterministic FSM 

First, we need to define the NFA using the methods defined above. We do this by defining a regular expression, and convert that regex into an FSM using `convertRegExToFSM()`. 

In [3]:
E4 = Choice(Conc(Conc('a', Star('a')), 'b'), Conc(Conc('a', Star('a')), 'c')); A4 = convertRegExToFSM(E4); A4

0
3 6
2 b → 3
4 c → 6
1 b → 3
0 a → 1
5 c → 6
1 a → 2
0 a → 4
5 a → 5
2 a → 2
4 a → 5

###### Defining the Deterministic FSM 

Next, we need to define the equivalent DFA for the above NFA. This is done by inputting the NFA into the method `deterministicFSM()`. We then split the DFA into a list of strings. 

In [4]:
A4det = deterministicFSM(A4); A4det = A4det.splitlines()
for i in A4det: 
    print(i)
    
# start state = 0 
# final states = 3, 6
# transitions = a, b, c 
# all states = 0, 1, 2, 3, 4, 5, 6 

{0}
{3} {6}
{1, 4} a → {5, 2}
{0} a → {1, 4}
{5, 2} a → {5, 2}
{5, 2} b → {3}
{1, 4} c → {6}
{1, 4} b → {3}
{5, 2} c → {6}


In the section below, we introduce the additional classes and methods needed to produce a Minimized Finite State Machine. These methods were referenced from the lecture notes (02 Regular Languages).

In [5]:
def deterministicFSM(fsm: FiniteStateMachine) -> FiniteStateMachine:
    qq0 = string({fsm.q0})
    QQ, RR, visited = {qq0}, set(), set()
    #print(QQ, RR, visited)
    while visited != QQ:
        qq = (QQ - visited).pop(); visited |= {qq}
        for t in fsm.T:
            rr = {r for (q, u, r) in fsm.R if u == t and q in qq}
            if rr != set(): QQ |= {string(rr)}; RR |= {(qq, t, string(rr))}
        #print(QQ, RR, visited)
    FF = {qq for qq in QQ for f in fsm.F if f in qq}
    return FiniteStateMachine(fsm.T, QQ, RR, qq0, FF)


def minimizeFSM(fsm: FiniteStateMachine) -> FiniteStateMachine:
    nxt = {(q, a): r for (q, a, r) in fsm.R}
    dist = {(q, r) for q in fsm.Q for r in fsm.Q if q != r and (q in fsm.F) != (r in fsm.F)}
    done = False #; print('initially ', dist)
    while not done:
        done = True
        for q in fsm.Q:
            for r in fsm.Q:
                if q != r and (q, r) not in dist and any(((q, u) in nxt) != ((r, u) in nxt) or \
                    ((q, u) in nxt) and ((nxt[(q, u)], nxt[(r, u)]) in dist) for u in fsm.T):
                    dist |= {(q, r)}; done = False #; print('adding ', q, r)
        #print('updated ', dist)
    # construct minimal FSM with frozensets as states
    QQ = {frozenset({q} | {r for r in fsm.Q if (q, r) not in dist}) for q in fsm.Q}
    RR = {(qq, u, rr) for qq in QQ for rr in QQ for u in fsm.T if any((q, u, r) in fsm.R for q in qq for r in rr)}
    qq0 = {qq for qq in QQ if any(q in fsm.q0 for q in qq)}.pop()
    FF = {qq for qq in QQ if (qq & fsm.F) != set()}
    # convert frozensets into strings
    QQ = {string(qq) for qq in QQ}
    RR = {(string(qq), u, string(rr)) for (qq, u, rr) in RR}
    qq0 = string(qq0)
    FF = {string(qq) for qq in FF}
    output = FiniteStateMachine(fsm.T, QQ, RR, qq0, FF)
    output = str(output)
    return output


###### Defining the Minimized FSM 

Finally, we minimize the DFA using the method `minimizeFSM()`.

In [6]:
print("Deterministic FSM:")
for i in range(len(A4det)): 
    A4det[i] = A4det[i]
    print(A4det[i])
A4DET2 = deterministicFSM(A4);

print("Minimized FSM:")
A4min = minimizeFSM(A4DET2); A4min = A4min.splitlines()
for i in A4min:
    print(i)

Deterministic FSM:
{0}
{3} {6}
{1, 4} a → {5, 2}
{0} a → {1, 4}
{5, 2} a → {5, 2}
{5, 2} b → {3}
{1, 4} c → {6}
{1, 4} b → {3}
{5, 2} c → {6}
Minimized FSM:
{{0}}
{{3}, {6}}
{{5, 2}, {1, 4}} c → {{3}, {6}}
{{5, 2}, {1, 4}} b → {{3}, {6}}
{{5, 2}, {1, 4}} a → {{5, 2}, {1, 4}}
{{0}} a → {{5, 2}, {1, 4}}


### Animating the Minimization of Finite State Machine:

In this stage, we define a method for animating the minimization of an FSM.<br> 
The method `convertMIN(DFA, MIN)` takes two input values, both of which are lists of lists. It iterates through both lists, converting each element in the DFA list to the appropriate element in the MIN list. While converting each element, the method makes the corresponding changes to the animation. 

In [7]:
def convertMIN(DFA,MIN):
    eventDFA = [] # order of events for DFA
    eventMIN = [] # order of events for MIN
    new_stateMIN = [] # list of new states in MIN list
    new_stateDFA = [] # list of new states in DFA list
    
    for i in range(2,len(DFA)):
        oldDFA = DFA[i][DFA[i].find("{")+1:DFA[i].find("}")] # old state 
        end_bracket = (DFA[i].find("}")) # index of first end bracket 
        transitionDFA = DFA[i][end_bracket+2] # transition 
        newDFA = DFA[i][end_bracket+6:len(DFA[i])].replace('{','').replace('}','') # new state
        # prints model of DFA
        ga_det.add_edge(oldDFA,newDFA)
        ga_det.label_edge(oldDFA,newDFA,transitionDFA)
        eventDFA.append([oldDFA,transitionDFA, newDFA]) # states and transitions in DFA
        new_stateDFA.append(newDFA)
    ga_det.next_step()
    
    for i in range(2,len(MIN)):
        oldMIN = (MIN[i][MIN[i].find("{")+1:MIN[i].find("→")-4]).replace('{','').replace('}','')
        transitionMIN = MIN[i][MIN[i].find("→")-2] 
        newMIN = (MIN[i][MIN[i].find("→")+2:]).replace('{','').replace('}','') 
        eventMIN.append([oldMIN,transitionMIN,newMIN])
        new_stateMIN.append(newMIN)
    eventDFA = sorted(eventDFA)
    eventMIN = sorted(eventMIN)
    
    # iterate through both lists, converting DFA -> MIN
    while (eventDFA != eventMIN): # end when both lists are equal 
        for i in eventDFA:
            for j in eventMIN:
                if i[0] in j[0] and i[1]==j[1]: # if oldDFA in MIN 
                    index = (eventDFA.index(i)) # index of value to be replaced with combined state 
                    ga_det.remove_edge(i[0],i[2]) # remove old edge 
                    eventDFA[index] = j # replace newDF A with newMIN
                    ga_det.add_edge(j[0],j[2]) # add new edge
                    ga_det.label_edge(j[0],j[2],j[1])
                    ga_det.next_step() 
        eventDFA = set(map(tuple, eventDFA))
        eventDFA = sorted(list(map(list, eventDFA)))
        eventMIN = sorted(eventMIN)

    # remove unnecessary nodes 
    new_stateDFA = list(set(new_stateDFA))
    for i in new_stateDFA:
        if i not in new_stateMIN:
            ga_det.remove_node(i)
            ga_det.next_step()
    
convertMIN(A4det, A4min)

### Running the Animation of the Minimization of Finite State Machine:

Here, we call the method `interactive()`, which was imported with the module `gvanim`. This method generates the interactive animation using the previously defined methods, `convertMIN()`.

Move the slider from left to right to view the animation of the minimization of the DFA.

Note: In this animation, the node for '6,3' is used twice, to represent the transitions '4,1,5,2 c → 1' and '4,1,5,2 b → 1'. The edges overlap, so it appears as if only one edge is displayed. 

In [8]:
interactive(ga_det,600)

interactive(children=(IntSlider(value=0, description='n', max=12), Output()), _dom_classes=('widget-interact',…